<a href="https://colab.research.google.com/github/Kaisano/gcollab-tor2jellyfin/blob/main/jellyfin-tor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Install Dependencies

In [1]:
!python -m pip install --upgrade pip setuptools wheel
%pip install libtorrent lbry-libtorrent dropbox tomlkit

!curl https://rclone.org/install.sh | sudo bash
%pip install rclone-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 65.6 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.2.0
    Uninstalling setuptools-75.2.0:
      Successfully uninstalled setuptools-75.2.0
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 68.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 58.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.1/572.1 kB 21.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [dropbox]
  % Total    

# 2. Setup
## 2.1 Create filesystem

In [2]:
import sys
import os, subprocess, glob
from concurrent.futures import ThreadPoolExecutor, Future, as_completed

import logging
import time
from timeit import default_timer as timer

from rich.progress import Progress, TextColumn, BarColumn, TaskProgressColumn, TimeRemainingColumn
from pathlib import Path

from tomlkit import document, table, nl, comment
from tomlkit import dumps
from tomlkit.toml_file import TOMLFile

from configparser import ConfigParser
from pathlib import Path

import libtorrent as lt
from rclone_python import rclone

from google.colab import drive
drive.mount('/content/drive')
import ipywidgets as widgets

# generate file to add
magnet_path = Path("/content/magnet.links")
magnet_file = TOMLFile(magnet_path.as_posix())

if not magnet_path.exists():
  magnet_document = document()

  example_folder_tor = table()
  example_folder_tor["category"] = "tv_shows"
  example_folder_tor["magnets"] = ["mag1"]

  example_movie_tor = table()
  example_movie_tor["category"] = "movies"
  example_movie_tor["magnets"] = ["mag2"]

  example_multifiles = table()
  example_multifiles["category"] = "anime"
  example_multifiles["magnets"] = ["mag3", "mag4", "mag5", "..."]

  magnet_document.add(comment("################################################################################"))
  magnet_document.add(comment("#  This file explicitly outlines how torrents should be stored.                #"))
  magnet_document.add(comment("################################################################################"))
  magnet_document.add(comment("the category should match the containing folder name in jellyfin. for example: category=/your/path/to/jellyfinmedia/[category]"))
  magnet_document.add("awesome_tv_show", example_folder_tor)
  magnet_document.add("exciting_movie", example_movie_tor)
  magnet_document.add(nl())
  magnet_document.add(comment("seperate file episodes"))
  magnet_document.add("cool_anime", example_multifiles)

  magnet_file.write(magnet_document)

Mounted at /content/drive


## Configure app

In [6]:
# observed that google has this file and copy that config file
gdrive_rclone_savepath = Path("/content/drive/MyDrive/Apps/jellyfin-tor/rclone/rclone.conf")
rclone_path = Path("/root/.config/rclone/rclone.conf")

# check if the config file exists in google drive
if gdrive_rclone_savepath.exists():
  rclone_path.parent.mkdir(parents=True, exist_ok=True)
  !cp {gdrive_savepath.as_posix()} {rclone_path.as_posix()}

# first time setup if the config file doesn't exist in the cloud
if not rclone_path.exists():
  rclone_path.parent.mkdir(parents=True, exist_ok=True)
  rclone_path.touch()

  # generate the rclone config
  rclone_cfg = ConfigParser()
  rclone_cfg['dropbox'] = \
  {
      "type": "dropbox",
      "token": ""
  }

  # get refresh token from user host
  token = input(
  """
  With rclone installed on a local system outside of google colab,
  run \"rclone authorize dropbox\" and paste the result here:
  """)
  rclone_cfg['dropbox']['token'] = token

  # create the rclone config file
  with rclone_path.open('w') as cfgwriter:
    rclone_cfg.write(cfgwriter)

With rclone installed on a local system outside of google colab, run "rclone authorize dropbox" and paste the result here: {"access_token":"sl.u.AGJ_iMqJnREnFSLTVAK7WqJAIoZI6NP3VN39ycRv6dbR0tdcDbk7JAS4XDD1AqbZ38d5n1zdImcotf5FCnJlqVEc2bXRmL4Ozi7ZKvv9OoSJg4nfjj9ZbwWS0-PQwCyWW42AuFVKjCUSmBYPW97kifh-3-Sx_pMBW8yaWAujFt3VW2iPanTxT0xgIKFzvPbh3wnMeWw80X6dN7aWCuQ3FIxiMFN90AWkw7jRGI7WrYo2I1CZW_LeTfrGJ1S_uIQq2E5nEyX9UCTMh9p9TkPM-dwjqit4z8LbcyijSdROfNiwOs5q6yCyyp3uTYZ72XSB19Y72Khz4mJYWEyDQ27HpSBq2jsiF8miZYpu53wAE3ED2Pl3mfIW9aKnmQ9AQ7lJviWqA73gKRjeSblVW1c0AotO_uF6XN2zMhdmgU1zZOVuGY9Qz1E_Zo-5tYenKQlmN6Gumj7WC9MJ15PGvZDleJxPZ73S5hNham_BgbTyvGMihoPpJVb37VFH1q3hqsCZ6hmrVoCWC0e-47thjbgUXj1TUlEE87gsH76KPUopyX6VOqln3H_Kxzn0Dm2eWI4Fechvn4150zuBYjKG2bE2Siy49pVRMivQ5_Q0NvZBCFG220w6hH32lAo38yvi5sf2xldG6XKphCZLXBMwkmsgneNSXi36yZ7BYzhjkHj99bY_KarCmGqv4sTYx8kDmDByc6HQcarCpUnp5aQ5m5vGQ9AYgIgbvaoFAXfY7FvNnTd0-sFkS757kGOn0iwhvP0wSTFDmdadqt5VRkGTaRblIRlaaKVyHftgaXG7OSf7zhRTKHThd8w2paIvBU68eDkc4RyFcbkh2gjA_4jGQSmDYXq

In [10]:
# get app config
app_conf_path = Path("/content/drive/MyDrive/Apps/torrent_dl/jellyfin-tor.conf")

app_cfg = ConfigParser()
if not app_conf_path.exists():
  app_conf_path.parent.mkdir(parents=True, exist_ok=True)
  app_conf_path.touch()

  app_cfg['Jellyfin'] = \
  {
      "media_folder": "",
  }

  with app_conf_path.open('w') as cfgwriter:
    app_cfg.write(cfgwriter)

else:
  app_cfg.read(app_conf_path.as_posix())

KeyboardInterrupt: Interrupted by user

# 3. Config app

In [94]:
# --- Row template as a class ---
class LinkWidget:
    def __init__(self, parent_box):
      self.parent_box = parent_box

      self.btn_up = widgets.Button(description='↑', button_style='info', layout=widgets.Layout(width="2%"))
      self.btn_down = widgets.Button(description='↓', button_style='info', layout=widgets.Layout(width="2%"))
      self.text = widgets.Text(placeholder='magnet:?xt=urn:btih:', layout=widgets.Layout(width="50%"))
      self.btn_add = widgets.Button(description='+', button_style='success', layout=widgets.Layout(width="2%"))
      self.btn_remove = widgets.Button(description='-', button_style='danger', layout=widgets.Layout(width="2%"))

      self.widget = widgets.HBox([self.btn_up, self.btn_down, self.text, self.btn_add, self.btn_remove])

      # wire button actions
      self.btn_add.on_click(self.add_row)
      self.btn_remove.on_click(self.remove_row)
      self.btn_up.on_click(self.move_up)
      self.btn_down.on_click(self.move_down)

      self.links = list()

    # --- button actions ---
    def add_row(self, _):
      rows = list(self.parent_box.children)
      idx = rows.index(self.widget)
      new_row = LinkWidget(self.parent_box).widget

      # insert right after this row
      rows.insert(idx + 1, new_row)

      self.parent_box.children = rows


    def remove_row(self, _):
      new_children = list(self.parent_box.children)
      if(len(new_children)) <= 1:
        return
      new_children.remove(self.widget)
      self.parent_box.children = new_children

    def move_up(self, _):
      rows = list(self.parent_box.children)
      idx = rows.index(self.widget)
      if idx > 0:
          rows[idx], rows[idx - 1] = rows[idx - 1], rows[idx]
          self.parent_box.children = rows

    def move_down(self, _):
      rows = list(self.parent_box.children)
      idx = rows.index(self.widget)
      if idx < len(rows) - 1:
          rows[idx], rows[idx + 1] = rows[idx + 1], rows[idx]
          self.parent_box.children = rows

    def obtain_link(self):
      self.links.clear()
      for hbox in self.parent_box.children:
        link = hbox.children[2].value
        if link == "":
          continue

        self.links.append(hbox.children[2].value)

In [115]:
class MediaModule:
  def __init__(self):

    self.title_textbox = widgets.Text(description="Title", placeholder="Awesome Movie Title", layout=widgets.Layout(width="30%"))
    self.category_textbox = widgets.Text(description="Category", placeholder="movies, tv_shows, anime, ...", layout=widgets.Layout(width="30%"))

    self.link_list = widgets.VBox([])
    self.initial_row = LinkWidget(self.link_list)
    self.link_list.children = [self.initial_row.widget]

    self.widget = widgets.VBox([self.title_textbox, self.category_textbox, self.link_list])

In [119]:
class MediaModuleManager:
  def __init__(self):
    self.accordion = widgets.Accordion(children=[])
    self.modules = []

    # Add first module
    self.add_module()

    # Button to add more
    self.add_button = widgets.Button(
        description="Add Module",
        icon="plus",
        button_style="info",
        layout=widgets.Layout(width="20%")
    )
    self.add_button.on_click(self.add_module)

    self.submit_button = widgets.Button(
        description="Submit",
        button_style="success",
        icon='check',
        layout=widgets.Layout(width="20%")
    )
    self.submit_button.on_click(self.submit)

    self.widget = widgets.VBox([self.add_button, self.accordion, self.submit_button])


  def add_module(self, _=None):
    module = MediaModule()
    module.title_textbox.observe(lambda change, idx=len(self.modules):
                             self.accordion.set_title(idx, change['new']),
                             names='value')
    self.modules.append(module)

    # insert module widget into accordion
    children = list(self.accordion.children)
    children.append(module.widget)
    self.accordion.children = children

    # update titles
    self.accordion.set_title(len(self.modules)-1, f"Series {len(self.modules)}")

    # auto-open newest section
    self.accordion.selected_index = len(self.modules) - 1

  def submit(self, _):
    print("=== Submitted Data ===")
    for i, module in enumerate(self.modules):
        # fetch title + category
        title = module.title_textbox.value
        category = module.category_textbox.value

        # fetch links
        links = []
        for row in module.link_list.children:
            link = row.children[2].value
            if link.strip():
                links.append(link)

        print(f"\nModule {i+1}")
        print("Title:", title)
        print("Category:", category)
        print("Links:")
        for link in links:
            print(" •", link)


In [120]:
manager = MediaModuleManager()
display(manager.widget)

=== Submitted Data ===

Module 1
Title: test
Category: 
Links:

Module 2
Title: ser
Category: 
Links:

Module 3
Title: 
Category: 
Links:

Module 4
Title: 
Category: 
Links:


# Download torrent